In [ ]:
!pip install -U transformers
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 71.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 30.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from huggingface_hub import login
login()

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 5.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-1.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-Coder-1.5B-Instruct", load_in_4bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
from datasets import Dataset
import torch

# Load the dataset from CSV
dataset = Dataset.from_csv("/content/Query_response_with_responses.csv")
column_names = ['Unnamed: 0', 'article', 'abstract', 'section_names', 'query', 'response']
# Add special tokens if necessary
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


# Define preprocess function
def preprocess_function(examples):
    inputs = [
        f"### Context:\n{context}\n\n### Query:\n{query}\n\n"
        for context, query in zip(examples["article"], examples["query"])
    ]
    tokenized = tokenizer(inputs,  truncation=True, padding="max_length", max_length=512)
    responses  = [f"### Response: \n{response}" for response in examples["response"]]
    tokenized_outputs = tokenizer(responses, truncation=True, padding = "max_length", max_length=512)
    input_ids = tokenized["input_ids"]
    labels = tokenized_outputs["input_ids"]

    # Mask all non-response tokens in labels
    # for i, input_ids_seq in enumerate(input_ids):
    #     response_start = inputs[i].find("### Response:\n") + len("### Response:\n")
    #     response_token_start = len(tokenizer.encode(inputs[i][:response_start], add_special_tokens=False))

    #     labels[i][:response_token_start] = [-100]  # Mask everything before the response
    # Mask padding tokens in labels
    for i in range(len(labels)):
        labels[i] = [
            token if token != tokenizer.pad_token_id else -100
            for token in labels[i]
        ]

    tokenized["labels"] = labels
    return tokenized

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=column_names)

train_test_split = tokenized_datasets.train_test_split(test_size=0.2)

train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

In [ ]:
inputs = {
    key: torch.tensor(train_dataset[0][key]).unsqueeze(0).to("cuda")  # Add batch dimension
    for key in ["input_ids", "attention_mask", "labels"]
}
print(f"Labels shape: {inputs['labels'].shape}")
print(f"Labels dtype: {inputs['labels'].dtype}")

Labels shape: torch.Size([1, 512])
Labels dtype: torch.int64


In [ ]:
# Define the QLoRA configuration
lora_config = LoraConfig(
    r=8,  # Reduced rank for smaller models
    lora_alpha=16,  # Adjusted scaling
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Verify these with your model
    lora_dropout=0.05,  # Lower dropout for smaller models
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply QLoRA to the model
model = get_peft_model(model, lora_config)
model.train()

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    load_best_model_at_end=True# Use mixed precision if supported
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    peft_config=lora_config,
    max_seq_length=1024,
    packing=True,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./qwen-1.5b-qlora-finetuned")
tokenizer.save_pretrained("./qwen-1.5b-qlora-finetuned")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in t

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,7.746000,5.906114
2,3.820900,3.805172
3,2.885600,2.987012
4,2.167600,2.454268
5,1.802100,2.159137
6,1.743300,1.922664
7,1.527300,1.750164
8,1.453800,1.665784
9,1.357100,1.573359
10,1.105500,1.560871


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered i

('./qwen-1.5b-qlora-finetuned/tokenizer_config.json',
 './qwen-1.5b-qlora-finetuned/special_tokens_map.json',
 './qwen-1.5b-qlora-finetuned/vocab.json',
 './qwen-1.5b-qlora-finetuned/merges.txt',
 './qwen-1.5b-qlora-finetuned/added_tokens.json',
 './qwen-1.5b-qlora-finetuned/tokenizer.json')

In [1]:
import torch

torch.cuda.empty_cache()
del model
del trainer

import gc
gc.collect



NameError: name 'model' is not defined

In [ ]:
# Install Required Libraries

import locale
locale.getpreferredencoding = lambda: "UTF-8"
!apt-get install git-lfs

# Login to Hugging Face
from huggingface_hub import notebook_login

notebook_login()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Create and Clone a Hugging Face Repository
from huggingface_hub import create_repo, Repository

# Replace with your Hugging Face username and desired repository name
repo_name = "qwen-1.5b-finetuned-query-response"
username = "yugdave"

#create_repo(repo_name, token=True)

# Clone the repository
repo = Repository(local_dir="/content/qwen-1.5b-finetuned-query-response", clone_from=f"{username}/{repo_name}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/yugdave/qwen-1.5b-finetuned-query-response into local empty directory.


In [ ]:
!cp -r /content/qwen-1.5b-qlora-finetuned/* /content/qwen-1.5b-finetuned-query-response/

In [ ]:
!git config --global user.email "yugdave578@utexas.edu"
!git config --global user.name "yugdave"


In [ ]:
repo.push_to_hub(commit_message="Add my fine-tuned model")

Upload file tokenizer.json:   0%|          | 32.0k/10.9M [00:00<?, ?B/s]

Upload file adapter_model.safetensors:   0%|          | 32.0k/8.34M [00:00<?, ?B/s]

To https://huggingface.co/yugdave/qwen-1.5b-finetuned-query-response
   5e5c77b..c733641  main -> main

   5e5c77b..c733641  main -> main



'https://huggingface.co/yugdave/qwen-1.5b-finetuned-query-response/commit/c733641c47ddaac07062ae1cda86a41667d9a86f'